In [20]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entire‰ly on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import time

## Group Assignment
### Team Number: 04
### Team Member Names: Arav Talati, Iris Hu, Nyra Rodrigues
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [21]:
stock_csv = "Tickers_Example.csv"

# Read the stock symbols from CSV
stocks = pd.read_csv(stock_csv, header=None)
stocks.columns = ['Column1']

start_date = "2023-10-01"
end_date = "2024-09-30"

stock_list = []

# Loop through each ticker
for tckr in stocks["Column1"]:
    stock = yf.Ticker(tckr)

    # Check if the stock has 'currency' in the info
    if "currency" in stock.info.keys():
        stock_currency = stock.info["currency"]
        
        # Get the historical data for the ticker with monthly intervals
        stock_hist = stock.history(start=start_date, end=end_date, interval="1mo")
        time.sleep(0.3)

        # Calculate the average volume over the valid months
        if not stock_hist.empty:  # If there's data after dropping NaN volumes
            stock_volume = stock_hist['Volume'].dropna().mean()

            # Check if currency is USD or CAD and if the volume is above 100,000
            if (stock_currency == "USD" or stock_currency == "CAD") and stock_volume > 100000:
                stock_list.append(tckr)

# Final list of tickers that meet the criteria
stock_list


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [22]:
start_date = "2024-09-01"
end_date = "2024-11-01"

MarketIndex1 = "^GSPTSE"  # TSX 60 #confirm ticker
MarketIndex2 = "^GSPC"  # S&P 500


market_hist1 = yf.Ticker(MarketIndex1).history(start=start_date, end=end_date)['Close']
market_hist2 = yf.Ticker(MarketIndex2).history(start=start_date, end=end_date)['Close']

market_hist1["Returns"] = market_hist1.pct_change() * 100
market_hist2["Returns"] = market_hist2.pct_change() * 100

market_avg_returns = (market_hist1["Returns"] + market_hist2["Returns"]) / 2

market_var = market_avg_returns.var()

# get_stock_beta returns the beta of the ticker inputted
def beta_val(stock_returns):
    df = pd.DataFrame({"Market Returns": market_avg_returns, "Stock Returns": stock_returns}).dropna()
    covariance = df.cov().iloc[0, 1]
    return covariance / market_var


#betas_list takes a list of tickers and outputs a list of all the betas of every ticker in the list
def betas_list(ticker_list):
    betas = {}
    for ticker in ticker_list:
        stock_hist = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
        time.sleep(0.3)
        ticker_returns = stock_hist.pct_change()*100 
        betas[ticker] = beta_val(ticker_returns)
    return betas


all_betas = betas_list(stock_list)

In [23]:
beta_dict = betas_list(all_betas.keys())  #change based on taken
beta_df = pd.DataFrame.from_dict(beta_dict, orient='index')
beta_df.columns = ['Beta']
beta_df.index.name = 'Ticker'

In [44]:
#Sorts the dictionary from lowest beta to highest beta
sorted_beta_df = beta_df.sort_values('Beta').copy()

#Creates a dataframe with the 10 highest betas
#eligible_stocks = sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1]

if (len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) < 12):
    eligible_stocks = sorted_beta_df.iloc[-15:]
elif ((len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) > 30)): 
    eligible_stocks = sorted_beta_df.iloc[-30:] 
else:
    eligible_stocks = sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1]
    
eligible_stocks

,Beta
Ticker,
AAPL,1.111248
TXN,1.156412
BIIB,1.189811
USB,1.267148
LLY,1.289998
BAC,1.327827
AIG,1.410103
CAT,1.552065
AXP,1.606705


In [45]:
def calculate_sharpe_ratio(ticker, risk_free_rate):
    stock_data = yf.Ticker(ticker).history(start="2024-09-01", end="2024-11-01")['Close']
    time.sleep(0.3)
    stock_returns = stock_data.pct_change()
    average_return = stock_returns.mean()
    std_deviation = stock_returns.std()
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio


sharpe_ratios = {}
for ticker in eligible_stocks.index:
    sharpe_ratios[ticker] = calculate_sharpe_ratio(ticker, 0)


for ticker, sharpe_ratio in sharpe_ratios.items():
    eligible_stocks.loc[ticker, "Sharpe Ratio"] = sharpe_ratio

eligible_stocks = eligible_stocks.sort_values('Sharpe Ratio', ascending=False)

if (len(eligible_stocks[eligible_stocks['Sharpe Ratio'] > 0]) < 12):
    eligible_stocks = eligible_stocks.iloc[:12]
else:
    lowSharpe = eligible_stocks[eligible_stocks['Sharpe Ratio'] <= 0].index
    eligible_stocks.drop(lowSharpe, inplace=True)

eligible_stocks

,Beta,Sharpe Ratio
Ticker,,
SHOP.TO,2.497989,0.164786
CAT,1.552065,0.163863
PYPL,1.674955,0.128491
AXP,1.606705,0.103976
AMZN,1.891268,0.092459
USB,1.267148,0.084439
BAC,1.327827,0.064708
C,1.679481,0.062344
BB.TO,2.694406,0.038978


In [46]:
#eligible_stocks["Projected Returns"] = 0
for tckr in eligible_stocks.index:
    try:
        ticker = yf.Ticker(tckr)
        ticker_hist = ticker.history(start=start_date, end=end_date)
        time.sleep(0.3)
        ticker_hist['EMA Short'] = ticker_hist['Close'].ewm(span=2, adjust=True, min_periods=5).mean()
        ticker_hist['EMA Long'] = ticker_hist['Close'].ewm(span=50, adjust=True, min_periods=5).mean()
        if ticker_hist.empty:
            eligible_stocks.loc[tckr, 'Projected Returns'] = -10000
        eligible_stocks.loc[tckr, 'Projected Returns'] = ticker_hist['EMA Short'].iloc[len(ticker_hist)-1] - ticker_hist['EMA Long'].iloc[len(ticker_hist)-1]
    except:
        pass

eligible_stocks = eligible_stocks.sort_values('Projected Returns', ascending=False)

if (len(eligible_stocks[eligible_stocks['Projected Returns'] > -2]) < 12):
    eligible_stocks = eligible_stocks.iloc[:12]
elif (len(eligible_stocks[eligible_stocks['Projected Returns'] > -2]) > 24):
    eligible_stocks = eligible_stocks.iloc[:24]
else:
    lowReturns = eligible_stocks[eligible_stocks['Projected Returns'] <= -2].index
    eligible_stocks.drop(lowReturns, inplace=True)


eligible_stocks.dropna(axis=0, inplace=True)
eligible_stocks

,Beta,Sharpe Ratio,Projected Returns
Ticker,,,
TXN,1.156412,0.026437,2.587035
USB,1.267148,0.084439,1.721775
C,1.679481,0.062344,1.572351
AMZN,1.891268,0.092459,1.218667
AXP,1.606705,0.103976,1.217195
BAC,1.327827,0.064708,0.921243
SHOP.TO,2.497989,0.164786,0.873995
PYPL,1.674955,0.128491,0.565620
AIG,1.410103,0.012834,0.526661


In [47]:
sector_etfs = {
    "Technology": "XLK",
    "Healthcare": "XLV",
    "Financial Services": "XLF",
    "Consumer Cyclical": "XLY",
    "Energy": "XLE",
    "Utilities": "XLU",
    "Consumer Defensive": "XLP",
    "Industrials": "XLB",
    "Real Estate": "XLRE",
    "Communication Services": "XLC"
}

# Function to get P/E ratio for sector ETFs
def get_sector_pe_data(sector_etfs):
    sector_pe_data = {}
    for sector, etf in sector_etfs.items():
        
        # Fetch ETF data
        sector_etf = yf.Ticker(etf)
        info = sector_etf.info
        
        # Get the P/E ratio of the sector ETF
        pe_ratio = info.get('trailingPE', None)
        
        if pe_ratio:
            sector_pe_data[sector] = pe_ratio
    
    return sector_pe_data

# Get sector P/E data for each ETF
sector_pe_data = get_sector_pe_data(sector_etfs)

sector_pe_data

{'Technology': 44.232452,
 'Healthcare': 24.40495,
 'Financial Services': 22.77223,
 'Consumer Cyclical': 33.559216,
 'Energy': 8.992434,
 'Utilities': 28.81466,
 'Consumer Defensive': 27.642971,
 'Industrials': 18.014141,
 'Real Estate': 33.516953,
 'Communication Services': 36.65689}

In [48]:
def get_stock_pe(tickers):
    stock_data = {}
    for ticker in tickers:
        
        stock = yf.Ticker(ticker)
        # Get stock info
        info = stock.info
        sector = info.get('sector', 'N/A')
        pe_ratio = info.get('trailingPE', None)  # trailing P/E ratio
        pb_ratio = info.get('priceToBook', None)
        
        # Add data to dictionary
        if (sector != 'N/A'): 
            if pe_ratio:
                stock_data[ticker] = {'sector': sector, 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': sector, 'PE': None}
        else:
            if pe_ratio:
                stock_data[ticker] = {'sector': 'N/A', 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': 'N/A', 'PE': None}

    return stock_data

stock_pe = get_stock_pe(eligible_stocks.index)

stock_pe

{'TXN': {'sector': 'Technology', 'PE': 36.908752},
 'USB': {'sector': 'Financial Services', 'PE': 15.810769},
 'C': {'sector': 'Financial Services', 'PE': 19.641026},
 'AMZN': {'sector': 'Consumer Cyclical', 'PE': 42.298508},
 'AXP': {'sector': 'Financial Services', 'PE': 21.570692},
 'BAC': {'sector': 'Financial Services', 'PE': 16.82971},
 'SHOP.TO': {'sector': 'Technology', 'PE': 99.206665},
 'PYPL': {'sector': 'Financial Services', 'PE': 20.291866},
 'AIG': {'sector': 'Financial Services', 'PE': 15.106361},
 'BB.TO': {'sector': 'Technology', 'PE': None},
 'AAPL': {'sector': 'Technology', 'PE': 37.523808},
 'CAT': {'sector': 'Industrials', 'PE': 18.077496}}

In [49]:
def compare_pe_to_sector(stock_data, sector_pe_data):
    # Print comparison of P/E ratios for each stock

    for tckr, data in stock_data.items():
        sector = data['sector']
        pe = data['PE']
        #pb = data['PB']
        
        if sector != 'N/A' and pe is not None:
            pe_sector_avg = sector_pe_data.get(sector, None)
            if pe_sector_avg is not None:
                eligible_stocks.loc[tckr, "comparisonPE"] = pe - pe_sector_avg
        else:
            eligible_stocks.loc[tckr, "comparisonPE"] = 0

compare_pe_to_sector(stock_pe, sector_pe_data)

eligible_stocks

,Beta,Sharpe Ratio,Projected Returns,comparisonPE
Ticker,,,,
TXN,1.156412,0.026437,2.587035,-7.323700
USB,1.267148,0.084439,1.721775,-6.961461
C,1.679481,0.062344,1.572351,-3.131204
AMZN,1.891268,0.092459,1.218667,8.739292
AXP,1.606705,0.103976,1.217195,-1.201538
BAC,1.327827,0.064708,0.921243,-5.942520
SHOP.TO,2.497989,0.164786,0.873995,54.974213
PYPL,1.674955,0.128491,0.565620,-2.480364
AIG,1.410103,0.012834,0.526661,-7.665869


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.